# 0 Init

In [ ]:
#DEZE HEB IK ZELF GEVONDEN! FFE AAN OWEN LATEN ZIEN!!!!
#https://huggingface.co/IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment?text=%E4%BB%8A%E5%A4%A9%E5%BF%83%E6%83%85%E4%B8%8D%E5%A5%BD
#info on fine tuning (from Owen!!!!) https://huggingface.co/transformers/v4.8.2/training.html CHECK THIS OUT! I CAN CHANGE THE NUMBER OF LABELS HERE TOO!!!!
##bertelec
#from transformers import BertForSequenceClassification
#from transformers import BertTokenizer
#import torch
#!pip install transformers
#tokenizer=BertTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')
#model=BertForSequenceClassification.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')

#text='其实我没想到他们会这样'

#output=model(torch.tensor([tokenizer.encode(text)]))
#print(torch.nn.functional.softmax(output.logits,dim=-1))

#tokens = tokenizer.encode("今天是我最快乐的一天！", return_tensors='pt')

#result = model(tokens)

#int(torch.argmax(result.logits))

In [ ]:
#ok first we run the finetuning tutorial from this one https://huggingface.co/transformers/v4.8.2/training.html
#then we replace the bert with the Chinese one, and the data with the WEIBO finetuning set
#then we put Torrent's data in there

In [ ]:
!pip install datasets
!pip install transformers
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Load Data and tokenize

## 1-1. Example dataset (IMDB)

### 1-1-1. Load Dataset

In [ ]:
# from datasets import load_dataset
# raw_datasets = load_dataset("imdb")

In [ ]:
# raw_datasets

### 1-1-1. Data Tokenize by Google BERT

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

## 1-2. Weibo Dataset

### 1-2-1 Load datast

In [ ]:
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/train/usual_train.xlsx
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/test/%E7%9C%9F%E5%AE%9E%E8%AF%84%E6%B5%8B%E9%9B%86/usual_test.xlsx


File ‘usual_train.xlsx’ already there; not retrieving.

File ‘usual_test.xlsx’ already there; not retrieving.



In [ ]:
map_dict = dict()

map_dict['angry']=0
map_dict['happy']=1
map_dict['neutral']=2
map_dict['surprise']=3
map_dict['sad']=4
map_dict['fear']=5

len(map_dict)

In [ ]:
import pandas as pd

In [ ]:
training_pd = pd.read_excel('/content/usual_train.xlsx',
                            names=['text', 'label']

training_pd['label'] = training_pd['label'].map(map_dict).astype('int')
training_pd.head(5)

,text,label
0,气死姐姐了，快二是阵亡了吗，尼玛，一个半小时过去了也没上车,0
1,妞妞啊，今天又承办了一个发文登记文号是126~嘻~么么哒~晚安哟,1
2,这里还值得注意另一个事实，就是张鞠存原有一个东溪草堂为其读书处。,2
3,这在前华约国家(尤其是东德)使用R-73的首次联合演习期间，被一些北约组织的飞行员所证实。,2
4,TinyThief上wii了？！,3


In [ ]:
# training_pd = training_pd[(training_pd['label'] == 0) | (training_pd['label'] == 1)]
training_pd = training_pd.dropna().reset_index(drop=True)

In [ ]:
testing_pd = pd.read_excel('/content/usual_train.xlsx',
                            names=['text', 'label'])

testing_pd['label'] = testing_pd['label'].map(map_dict).astype('int')
testing_pd.head(5)

,text,label
0,气死姐姐了，快二是阵亡了吗，尼玛，一个半小时过去了也没上车,0
1,妞妞啊，今天又承办了一个发文登记文号是126~嘻~么么哒~晚安哟,1
2,这里还值得注意另一个事实，就是张鞠存原有一个东溪草堂为其读书处。,2
3,这在前华约国家(尤其是东德)使用R-73的首次联合演习期间，被一些北约组织的飞行员所证实。,2
4,TinyThief上wii了？！,3


In [ ]:
# testing_pd = testing_pd[(testing_pd['label'] == 0) | (testing_pd['label'] == 1)]
testing_pd = testing_pd.dropna().reset_index(drop=True)

In [ ]:
from datasets import Dataset, DatasetDict

weibo_dict = {'train' : Dataset.from_pandas(training_pd),
                'test' : Dataset.from_pandas(testing_pd)}

weibo_ds = DatasetDict(weibo_dict)

### 1-2-2. Data Tokenize by wmmBERT

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-bert-wwm")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=102)

tokenized_datasets = weibo_ds.map(tokenize_function, batched=True)

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13721
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13721
    })
})

# 2. Fine-Tuning

In [ ]:
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
# full_train_dataset = tokenized_datasets["train"]
# full_eval_dataset = tokenized_datasets["test"]

## 2-1 Fine-tuning using Google BERT

In [ ]:
# from transformers import AutoModelForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
# from transformers import TrainingArguments

# training_args = TrainingArguments("test_trainer")

In [ ]:
# from transformers import Trainer

# trainer = Trainer(
#     model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
# )

In [ ]:
# trainer.train()

In [ ]:
# import numpy as np
# from datasets import load_metric

# metric = load_metric("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_train_dataset,
#     eval_dataset=small_eval_dataset,
#     compute_metrics=compute_metrics,
# )
# trainer.evaluate()

## 2-2 Fine-tuning using wmmBERT

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-bert-wwm", num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--hfl--chinese-bert-wwm/snapshots/ab0aa81da273504efc8540aa4d0bbaa3016a1bb5/config.json
Model config BertConfig {
  "_name_or_path": "hfl/chinese-bert-wwm",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "voca

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--hfl--chinese-bert-wwm/snapshots/ab0aa81da273504efc8540aa4d0bbaa3016a1bb5/pytorch_model.bin
Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initial

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(num_train_epochs=1,
                                  output_dir="output")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=full_train_dataset, 
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 13721
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1716
  Number of trainable parameters = 102269186


Step,Training Loss
500,0.351600
1000,0.270900
1500,0.228500


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1716, training_loss=0.27787376394916524, metrics={'train_runtime': 307.6319, 'train_samples_per_second': 44.602, 'train_steps_per_second': 5.578, 'total_flos': 719208930938760.0, 'train_loss': 0.27787376394916524, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 13721
  Batch size = 8


{'eval_loss': 0.09836030006408691,
 'eval_accuracy': 0.9720865826105969,
 'eval_runtime': 86.8076,
 'eval_samples_per_second': 158.062,
 'eval_steps_per_second': 19.768,
 'epoch': 1.0}